In [1]:
from geopy.geocoders import Nominatim
from osgeo import gdal, osr, ogr
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from bs4 import BeautifulSoup
import requests 
import shapefile
from shapely.geometry import Polygon
import pandas as pd
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import os

In [6]:
def clip_raster(raster_path, aoi_path, out_path, srs_id=4326, flip_x_y = False):
    """
    Clips a raster at raster_path to a shapefile given by aoi_path. Assumes a shapefile only has one polygon.
    Will np.floor() when converting from geo to pixel units and np.absolute() y resolution form geotransform.
    Will also reproject the shapefile to the same projection as the raster if needed.

    Parameters
    ----------
    raster_path
        Path to the raster to be clipped.
    aoi_path
        Path to a shapefile containing a single polygon
    out_path
        Path to a location to save the final output raster

    """
    # https://gis.stackexchange.com/questions/257257/how-to-use-gdal-warp-cutline-option
    with TemporaryDirectory() as td:
#         log.info("Clipping {} with {}".format(raster_path, aoi_path))
        raster = gdal.Open(raster_path)
        in_gt = raster.GetGeoTransform()
        srs = osr.SpatialReference()
        srs.ImportFromWkt(raster.GetProjection())
        intersection_path = os.path.join(td, 'intersection')
        aoi = ogr.Open(aoi_path)
        if aoi.GetLayer(0).GetSpatialRef().ExportToWkt() != srs.ExportToWkt():    # Gross string comparison. Might replace with wkb
#             log.info("Non-matching projections, reprojecting.")
            aoi = None
            tmp_aoi_path = os.path.join(td, "tmp_aoi.shp")
            reproject_vector(aoi_path, tmp_aoi_path, srs)
            aoi = ogr.Open(tmp_aoi_path)
        intersection = get_aoi_intersection(raster, aoi)
        min_x_geo, max_x_geo, min_y_geo, max_y_geo = intersection.GetEnvelope()
        if flip_x_y:
            min_x_geo, min_y_geo = min_y_geo, min_x_geo
            max_x_geo, max_y_geo = max_y_geo, max_x_geo
        width_pix = int(np.floor(max_x_geo - min_x_geo)/in_gt[1])
        height_pix = int(np.floor(max_y_geo - min_y_geo)/np.absolute(in_gt[5]))
        new_geotransform = (min_x_geo, in_gt[1], 0, max_y_geo, 0, in_gt[5])   # OK, time for hacking
        write_geometry(intersection, intersection_path, srs_id=srs.ExportToWkt())
        clip_spec = gdal.WarpOptions(
            format="GTiff",
            cutlineDSName=intersection_path+r"/geometry.shp",   # TODO: Fix the need for this
            cropToCutline=True,
            width=width_pix,
            height=height_pix,
            dstSRS=srs
        )
        out = gdal.Warp(out_path, raster, options=clip_spec)
        out.SetGeoTransform(new_geotransform)
        out = None 

In [8]:
clip_raster("/home/admin1/projects/BeCode/3D_houses/geo-files/temp/DSM/GeoTIFF/DHMVIIDSMRAS1m_k24.tif", "/home/admin1/projects/BeCode/3D_houses/geo-files/shapefiles/polygon.shp", "/home/admin1/projects/BeCode/3D_houses/geo-files/shapefiles/dsm.tif", srs_id=4326, flip_x_y = False)

NameError: name 'reproject_vector' is not defined

In [4]:
from tempfile import TemporaryDirectory

In [9]:
polygon =[[190246.80202307552, 224044.05999746546],
 [190239.8010630682, 224046.38294146955],
 [190237.82320706546, 224040.54242946208],
 [190220.13885505497, 224046.86998146772],
 [190209.9166470468, 224017.93942144886],
 [190209.28867904842, 224018.15689344704],
 [190206.45693504065, 224019.13762944937],
 [190203.0089990422, 224008.9510054402],
 [190214.9869830534, 224004.9109414406],
 [190216.19760704786, 224008.50031744316],
 [190239.6020230651, 224000.2600614354],
 [190243.74410306662, 224013.55900544673],
 [190251.1810310781, 224010.88598144427],
 [190254.2069510743, 224019.68802944943],
 [190247.3671430722, 224022.13807745278],
 [190252.07895107567, 224035.82102146],
 [190244.7435270697, 224038.15702146292],
 [190246.80202307552, 224044.05999746546]]

In [11]:
x,y= [x:x,y:y for x,y in polygon]

SyntaxError: invalid syntax (2099789960.py, line 1)